In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
dt = pd.read_excel("authorlist.xlsx",dtype="string")
#dt = dt.sort_values(by='Last Name\n')
len(dt)

In [ ]:
dt.columns

In [ ]:
from_list = pd.concat([dt[dt.columns[9]],dt[dt.columns[10]],dt[dt.columns[11]],dt[dt.columns[12]]])
for i in from_list.unique():
    print(i)

In [ ]:
for p in dt.iloc:
    if p['Present address (leave blank if your present address is already selected above)\n'] is not pd.NA:
        print(p['Present address (leave blank if your present address is already selected above)\n'])

In [ ]:
for p in dt.iloc:
    if p['Funding acknowledgement'] is not pd.NA:
        print(p['How would you like your name to appear (Generally initials and last name)\n'],":\t",p['Funding acknowledgement'])

In [ ]:
afflist=pd.read_csv("aff_list.csv")

In [ ]:
atmp=dt.iloc[21]['Select your primary institution\n'].strip()
info=afflist[afflist['orgAddress'].str.contains(atmp,case=False, na=False)]
afflist["used"]=False

In [ ]:
for p in dt.iloc:
    a = p[dt.columns[7]].strip()
    b=p['Last Name\n'].strip()
    c=a
    d=a
    if a.endswith(b):
        c=a.removesuffix(b)
        d=b
        
    else:
        print(b, " not in ", a)
        c= p['First Name'].strip()[0]+"."
        d= a.removeprefix(c).strip()
    print(c, ": ",d)

In [ ]:
import xml.etree.ElementTree as ET
author_tex=open("author.tex", "w")
author_tex.write("% !TeX root = main.tex\n")
arxiv_field=""
email_all=[]
root = ET.Element("collaborationauthorlist")
root.set("xmlns:foaf","http://xmlns.com/foaf/0.1/")
root.set("xmlns:cal","http://inspirehep.net/info/HepNames/tools/authors_xml/")
ET.SubElement(root, "cal:creationDate").text= str(datetime.datetime.now())
ET.SubElement(root, "cal:publicationReference").text="Final SeaQuest results on the flavor asymmetry of the proton light-quark sea with proton-induced Drell-Yan process"
col_e = ET.SubElement(root, "cal:collaborations")
col = ET.SubElement(col_e, "cal:collaboration")
col.set("id", "E906")
ET.SubElement(col,"foaf:name").text="FNAL E906/SeaQuest Collaboration"
ET.SubElement(col,"cal:experimentNumber").text="FNAL-E-906"
arxiv_field+="FNAL E906/SeaQuest Collaboration"

aff_e = ET.SubElement(root,"cal:organizations")

author_e = ET.SubElement(root,"cal:authors")
for p in dt.iloc:
    texline=[]
    author = ET.SubElement(author_e,"foaf:Person")
    fullname = p['First Name'].strip()
    #if not pd.isna(p['Middle Name\n']):
    #    fullname += " "+p['Middle Name\n'].strip()
    fullname += " "+p['Last Name\n'].strip()
    ET.SubElement(author,"foaf:name").text=fullname
    ET.SubElement(author,"foaf:givenName").text=p['First Name'].strip()
    ET.SubElement(author,"foaf:familyName").text=p['Last Name\n'].strip()
    ET.SubElement(author,"cal:authorNamePaper").text=p[dt.columns[7]].strip()

    first_initial = p[dt.columns[7]].strip()
    last_name = p['Last Name\n'].strip()
    last_paper = last_name
    if first_initial.endswith(last_name):
        first_initial=first_initial.removesuffix(last_name).strip()
        last_paper = last_name
    else:
        print(last_name, " not in ", first_initial)
        first_initial= p['First Name'].strip()[0]+"."
        last_paper= p[dt.columns[7]].strip().removeprefix(first_initial).strip()
        print ("Defaulting to: ", first_initial," and ",last_paper)
    ET.SubElement(author,"cal:authorNamePaperGiven").text=first_initial
    ET.SubElement(author,"cal:authorNamePaperFamily").text=last_paper
    
    
    texline.append("\\author{"+p[dt.columns[7]].strip().replace(" ","~")+"}")
    arxiv_field+=", " + p[dt.columns[7]]
    
    cnode = ET.SubElement(author, "cal:authorCollaboration")
    cnode.set("collaborationid","E906")
    if not pd.isna(p['position']):
        cnode.set("position",p['position'])
    aff = ET.SubElement(author, "cal:authorAffiliations")
   
    for col in range(9,12):
        if not pd.isna(p[dt.columns[col]]):
            atmp=p[dt.columns[col]].strip()
            info=afflist[afflist['option'].str.contains(atmp,case=False, na=False)]
            oid = ""
            address = ""
            if len(info.index)!=1:
                print(p)
            else:
                for i in info.index:
                    oid=afflist["id"][i]
                    address=afflist["orgAddress"][i]
                    afflist.loc[i,'used']=True
                ET.SubElement(aff,"cal:authorAffiliation").set("organizationid",oid)
                texline.append("\\affiliation{"+address+"}")
    if not pd.isna(p[dt.columns[14]]):
        atmp=p[dt.columns[14]].strip()
        info=afflist[afflist['id'].str.match(atmp,case=False, na=False)]
        oid = ""
        address = ""
        if len(info.index)!=1:
            print(p)
        else: 
            for i in info.index:
                oid=afflist["id"][i]
                address=afflist["orgAddress"][i]
                afflist.loc[i,'used']=True
            aff_al=ET.SubElement(aff,"cal:authorAffiliation")
            aff_al.set("organizationid",oid)
            aff_al.set("connection","Present Address")
            texline.insert(1, "\\altaffiliation[Present address: ]{"+address+"}")

    if not pd.isna(p['E-Mail Address\n']):
        email_address = p['E-Mail Address\n'].strip()
        email_all.append(email_address)
        if not pd.isna(p['position']) and p['position']=="Contact person":
            texline.insert(1,"\\email[Contact author: ]{"+email_address+"}")
    
    if not pd.isna(p[dt.columns[8]]):
        ids=ET.SubElement(author,"cal:authorids")
        id_e = ET.SubElement(ids, "cal:authorid")
        id_e.set("source", "ORCID")
        id_e.text=p[dt.columns[8]].strip()
        texline[0]=texline[0][:-1]+"\\,\\orcidlink{"+p[dt.columns[8]].strip()+"}}"
        
    for l in texline:
        author_tex.write(l+"\n")
    author_tex.write("\n")
for a in afflist.iloc:
    if (a['used']):
        aff = ET.SubElement(aff_e,"foaf:Organization")
        aff.set("id",a['id'])
        ET.SubElement(aff,"foaf:name").text=a['name']
        ror=ET.SubElement(aff, "cal:orgName")
        ror.set("source","ROR")
        ror.text=a['ror']
        if not pd.isna(a["INSPIRE"]):
            inspire=ET.SubElement(aff,"cal:orgName")
            inspire.set("source","INSPIRE")
            inspire.text=a["INSPIRE"]
        ET.SubElement(aff,"cal:orgAddress").text=a['orgAddress']
author_tex.write("\\collaboration{FNAL E906/SeaQuest Collaboration}\n\\noaffiliation")
author_tex.close()

In [ ]:
print(",".join(email_all))

In [ ]:
tree = ET.ElementTree(root)
ET.indent(tree)
with open('author.xml', 'wb') as f:
    f.write('<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE collaborationauthorlist SYSTEM "author.dtd">\n'.encode('utf8'))
    tree.write(f, 'utf-8')

In [ ]:
print(arxiv_field)